In [59]:
import pandas as pd
pd.set_option('max_colwidth', 30)
import re
import certifi
import urllib3
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [2]:
df = pd.read_csv("Openrice_Transformed_V2.csv")

In [54]:
df.head()

,Restaurant_ID,Restaurant_Name,District,Cuisine,Rating_Average,Cost_Average,Wait_Average,Reviews,Restaurant_Link
0,0,CHIRASHI,Wan Chai,Japanese,3.7,488.0,0.0,0Chirashi is a p...,www.openrice.com...
1,1,X,Causeway Bay,Chinese,3.6,60.0,0.0,There are a numb...,www.openrice.com...
2,2,10 Shanghai,Causeway Bay,Chinese,3.2,120.0,NaN,1957 & Company (...,www.openrice.com...
3,3,1935 Restaurant,Central,Chinese,4.8,270.0,NaN,The food is deli...,www.openrice.com...
4,4,1963,Wan Chai,Chinese,3.6,113.3,2.0,00This shop in W...,www.openrice.com...


In [7]:
# df.sort_values("Rating_Average",
#                      ascending = False)

### Tokenize

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = "english")

df["Reviews"] = df["Reviews"].fillna("")

tfidf_matrix = tfidf.fit_transform(df["Reviews"])

In [18]:
print(tfidf)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [11]:
# _stopwords = set(stopwords.words("english") + list(punctuation))

In [15]:
# https://intellipaat.com/community/32996/python-remove-stop-words-from-pandas-dataframe
# df["Reviews"].apply(lambda x: [i for i in x if i not in _stopwords])

In [19]:
tfidf_matrix.shape

(900, 21530)

In [20]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [21]:
cosine_sim[:2]

array([[1.        , 0.03966643, 0.06182234, ..., 0.02013431, 0.17051901,
        0.08445841],
       [0.03966643, 1.        , 0.08048414, ..., 0.02342414, 0.0413122 ,
        0.02158762]])

### Count Vectorizer

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words = "english")
count_matrix = count.fit_transform(df["Reviews"])

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

### Create Indices

In [41]:
# indices = df[["Restaurant_ID","Restaurant_Name"]]

indices = pd.Series(df.index,
                   index = df["Restaurant_Name"])

In [42]:
indices.head()

Restaurant_Name
 CHIRASHI          0
 X                 1
10 Shanghai        2
1935 Restaurant    3
1963               4
dtype: int64

In [63]:
def get_recommendations(title, cosine_sim = cosine_sim):
    
    # get the index of the movie that matches the title
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores,
                       key = lambda x: x[1],
                       reverse = True)
    
    sim_scores = sim_scores[1:11]
    
    rest_indices = [i[0] for i in sim_scores]
    
    return df[["Restaurant_Name","Restaurant_Link","District","Rating_Average","Wait_Average"]].iloc[rest_indices]

In [60]:
df.columns

Index(['Restaurant_ID', 'Restaurant_Name', 'District', 'Cuisine',
       'Rating_Average', 'Cost_Average', 'Wait_Average', 'Reviews',
       'Restaurant_Link'],
      dtype='object')

In [57]:
idx_test = indices["Five Guys"]
list(enumerate(cosine_sim[idx_test]))[:5]

[(0, 0.03309188590551711),
 (1, 0.019150869506771584),
 (2, 0.04684807810359639),
 (3, 0.03401687095562646),
 (4, 0.05059584837907506)]

In [64]:
get_recommendations("Five Guys", cosine_sim2)

,Restaurant_Name,Restaurant_Link,District,Rating_Average,Wait_Average
73,Burger Joys,www.openrice.com/en/hongko...,Wan Chai,4.1,2.0
611,Shake'em Buns,www.openrice.com/en/hongko...,Central,3.6,7.0
295,Honbo,www.openrice.com/en/hongko...,Wan Chai,3.4,3.0
95,Caliburger,www.openrice.com/en/hongko...,Wan Chai,2.6,3.0
734,The Butchers Club,www.openrice.com/en/hongko...,Wan Chai,3.1,10.0
75,Burgeroom,www.openrice.com/en/hongko...,Causeway Bay,2.8,7.0
71,Burger Circus,www.openrice.com/en/hongko...,Central,3.6,4.0
72,Burger Home,www.openrice.com/en/hongko...,Wan Chai,3.8,13.0
250,Gourmet Burger Union,www.openrice.com/en/hongko...,Central,2.8,2.0
58,Beef & Liberty,www.openrice.com/en/hongko...,Wan Chai,3.9,9.0
